# הסבר כללי על המחברת
מחברת זו עוסקת בטעינה ועיבוד של מערך הנתונים "$default$", ביצוע **נרמול** ($Standardization$) למשתני המודל, פיצול לסט אימון וסט בדיקה, הרצת אלגוריתם **$K-Nearest Neighbors (KNN)$** עם ערכי $k$ שונים, וכן ביצוע **$Cross-Validation$** ו-**$GridSearchCV$** כדי לאתר את $k$ הטוב ביותר.

נתאר בקצרה את השלבים:
1. **ייבוא ספריות וטעינת הנתונים** – נשתמש ב-$Pandas, Numpy, sklearn$ וכד'.
2. **בחירת משתני המשנה** – נפיק מתוך הדאטה רק את העמודות שחשובות לנו.
3. **י $Standardization$** – נרצה להפוך את המשתנים המספריים ($balance$ ו-$income$) לבעלי ממוצע 0 וסטיית תקן 1, כדי ש-$KNN$ לא יהיה מושפע מסקאלות שונות.
4. **י $Train/Test Split$** – נפריד את המידע שלנו לנתוני אימון ($train$) ונתוני בדיקה ($test$), לרוב 70%/30% או 80%/20%. כך נוכל להעריך את ביצועי המודל על נתונים שלא 'ראה' בזמן האימון.
5. **הרצת $KNN$** – נריץ את המודל עבור ערכי k שונים, ונבדוק את דיוק החיזוי ($accuracy$) על סט הבדיקה.
6. י **$Cross-Validation$** – נשתמש ב-5-$fold$ ו-10-$fold$ על כל המדגם כדי לקבל אומדן כללי של ביצועי המודל ולראות איזה $k$ נותן תוצאות עקביות.
7. י **$GridSearchCV$** – כלי אוטומטי שיבצע עבורנו חיפוש על $k$ אפשריים (או היפר פרמטרים אחרים), תוך שימוש ב-$CV$, ויידע אותנו איזה ערך $k$ מציג את הביצועים הטובים ביותר.


In [3]:
import pandas as pd
import numpy as np
from ISLP import load_data
# לדוגמה, אם יש לנו פונקציה ספציפית: default = loaddata('0default0')
# נשתמש בה. כאן נשתמש בהפקת נתונים דמה להדגמה.

df=load_data('default')  # If the ISLR2 package is not available, load data another way.

from sklearn.model_selection import train_test_split, GridSearchCV
# Convert data to a DataFrame
df = pd.DataFrame(df)

## שלב 3: בחירת משתני משנה - balance, income, default

In [4]:
subset = df[['balance', 'income', 'default']].copy()
subset.head()

,balance,income,default
0,729.526495,44361.625074,No
1,817.180407,12106.134700,No
2,1073.549164,31767.138947,No
3,529.250605,35704.493935,No
4,785.655883,38463.495879,No


## שלב 4: נרמול (Standardization) של המשתנים המספריים
נשתמש ב-**$StandardScaler$** משביליית $sklearn.preprocessing$ י

**למה אנחנו מבצעים ($Standardization$)  נרמול ?**
- כאשר משתמשים בשיטות מבוססות מרחק, כגון $K-Nearest Neighbors$, רצוי שכל המשתנים המספריים יהיו בסקאלה דומה, על מנת ש**אף משתנה לא ישפיע באופן מוגזם** רק בזכות הערכים הגדולים שלו.
- י $StandardScaler$ מבצע טרנספורמציה כך שלכל עמודה (משתנה) יהיה ממוצע 0 וסטיית תקן 1. זהו לרוב הצעד המקובל כשעובדים עם מודלים רגישים לסקאלה.


In [5]:
from sklearn.preprocessing import StandardScaler

X = subset[['balance', 'income']]  # משתני ניבוי
y = subset['default']             # משתנה המטרה

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled[:5]  # הצצה לחמשת השורות הראשונות לאחר הנרמול

array([[-0.21883482,  0.81318727],
       [-0.03761593, -1.60549572],
       [ 0.49241019, -0.13121225],
       [-0.6328925 ,  0.16403093],
       [-0.10279088,  0.37091513]])

## שלב 5: חלוקה לסט אימון וסט בדיקה

**למה מפצלים לסט אימון וסט בדיקה?**
- המטרה היא לאמן את המודל שלנו על חלק מהנתונים ולבדוק את איכות החיזוי שלו על נתונים חדשים שלא נראו באימון.
- כך אנחנו משיגים אומדן טוב יותר ליכולת הכללה ($Generalization$) של המודל על נתונים עתידיים.
- הפרמטר $test\_size=0.3$ מציין ש-30% מהנתונים ילכו לסט הבדיקה.
- י $stratify=y$ מאפשר לשמור על יחס דומה של $Yes/No$ (או של כל קטגוריות) בין סט האימון לסט הבדיקה, בעיקר חשוב אם יש חוסר איזון בכמות הדוגמאות.


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled,  # מאפיינים אחרי נרמול
    y,         # משתנה המטרה
    test_size=0.3,
    random_state=42,
    stratify=y  # כאשר יש חוסר איזון בין Yes/No, כדאי להשתמש ב-stratify
)

print("X_train shape:", X_train.shape)
print("X_test shape:\t", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:\t", y_test.shape)

X_train shape: (7000, 2)
X_test shape:	 (3000, 2)
y_train shape: (7000,)
y_test shape:	 (3000,)


## שלב 6: הרצת K-Nearest Neighbors עבור k = 1, 5, 20, 70

**מה זה KNN?**
- אלגוריתם $K-Nearest Neighbors$ (שכנים קרובים) מאחסן את כל נקודות האימון, ובזמן החיזוי לוקח את $k$ השכנים הכי קרובים לנקודה שחוזים לה.
- מסתכל על הקטגוריה (או הערך) הנפוץ ביותר בין אותם $k$ שכנים, ומשם גוזר את החיזוי.

**למה בודקים ערכים שונים של k?**
- ערך $k$ קטן (למשל 1) אומר שאנו סומכים רק על השכן הכי קרוב, שעשוי להיות רגיש לרעש.
- ערך $k$ גדול (למשל 70) מרכך השפעות של נקודה בודדת אבל עלול לערבב מידע מאזורים רחוקים.
- לכן, מקובל לנסות טווח ערכים ולראות איפה מקבלים את הביצועים הטובים ביותר.


In [7]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

k_values = [1, 5, 20, 70]
models = {}

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    models[k] = knn

    y_pred = knn.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"K={k}: דיוק על סט הבדיקה = {acc:.3f}")

K=1: דיוק על סט הבדיקה = 0.959
K=5: דיוק על סט הבדיקה = 0.968
K=20: דיוק על סט הבדיקה = 0.971
K=70: דיוק על סט הבדיקה = 0.971


**מה זה `accuracy_score`?**
- זוהי פונקציה שבודקת את אחוז הדגימות שחזינו עבורן נכון מתוך כלל הדגימות בסט הבדיקה.
- מחושבת כ: (מספר החיזויים הנכונים) / (סה"כ הדגימות).

## שלב 7: הרצת $Cross Validation (5-fold ו-10-fold)$ י

**מה זה $Cross Validation (CV)?$ י**
- י $CV$ היא שיטה בה מחלקים את כל המדגם למספר קיפולים ($folds$), לדוגמה 5 או 10. 
- ב-5-$fold CV$, מחלקים את הנתונים ל-5 תתי-קבוצות: בכל סיבוב משתמשים ב-4 קבוצות כסט אימון ובקבוצה ה-5 כסט בדיקה פנימי, ואז מחליפים.
- מקבלים כך 5 מדדים (למשל דיוק) – אחד לכל סיבוב – ומחשבים את הממוצע שלהם.
- זה נותן לנו הערכה יציבה יותר של ביצועי המודל, כי הוא נבדק על כל חלקי הדאטה.

**הפרמטר `cv`**
- כשאנו מציינים $cv=5$ אנחנו עושים$ 5-fold CV$. י
- כשאנו מציינים $cv=10$ אנחנו עושים$ 10-fold CV$. י

כך אנו רואים איך המודל עובד על חלקים שונים מהנתונים, במקום רק חלוקה בודדת לסט אימון וסט בדיקה.


In [8]:
from sklearn.model_selection import cross_val_score

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    
    # 5-fold CV
    scores_5 = cross_val_score(knn, X_scaled, y, cv=5)
    mean_5 = scores_5.mean()
    
    # 10-fold CV
    scores_10 = cross_val_score(knn, X_scaled, y, cv=10)
    mean_10 = scores_10.mean()
    
    print(f"K={k}: 5-fold CV={mean_5:.3f}, 10-fold CV={mean_10:.3f}")

K=1: 5-fold CV=0.957, 10-fold CV=0.956
K=5: 5-fold CV=0.970, 10-fold CV=0.969
K=20: 5-fold CV=0.972, 10-fold CV=0.973
K=70: 5-fold CV=0.972, 10-fold CV=0.972


**הסבר על תוצאת הפלט**
- בשורת ההדפסה רואים לדוגמה: $K=1: 5-fold CV=0.957, 10-fold CV=0.956$.י
- תוצאות קרובות מעידות על עקביות, ותוצאת $CV$ גבוהה מעידה שהמודל מסתדר היטב עם כל חלקי הדאטה.


# Use GridSearchCV to find the best k

**למה GridSearchCV?**
- י $GridSearchCV$ היא דרך לבצע **חיפוש על כל קומבינציות** של היפר-פרמטרים שהגדרנו (במקרה זה, ערכי $k$) תוך שימוש ב-$Cross Validation$. י 
- בכל ערך של $k$ היא תבצע אימון (על $Train folds$) ובדיקה (על $Validation fold$) ותמדוד את הביצועים.
- בסיום התהליך, $best\_params\_$ יחזיר את ערך $k$ שהשיג את הביצועים הטובים ביותר.
- $best\_score\_$ מציין את ציון הדיוק הממוצע הגבוה ביותר ב-$CV$ עבור הערך האידיאלי של $k$.


In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

# הנחה: הנתונים כבר חולקו ל-X_train, X_test, y_train, y_test
# (הכוללים את משתני הניבוי המנורמלים ואת המשתנה המטרה)

knn = KNeighborsClassifier()

param_grid = {'n_neighbors': [1, 5, 10, 20, 50, 70]}  # Define range of k
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Best k and corresponding accuracy
best_k = grid_search.best_params_['n_neighbors']
best_score = grid_search.best_score_

print(f"Best k: {best_k}")
print(f"Best cross-validation accuracy: {best_score:.2f}")

Best k: 10
Best cross-validation accuracy: 0.97



- כך אנחנו יודעים באיזה $k$ כדאי להשתמש כשנחזה על נתונים חדשים, על מנת למקסם את הסיכוי לדיוק מירבי.
- בשיטה זו, כאשר נרצה לעשות ניבוי סופי, נשתמש ב-$KNeighborsClassifier(n_neighbors=best_k)$ ונאמן אותו על מלוא סט האימון, ואז נחזה על הנתונים החדשים.
